# Astronomy 8824 - Numerical and Statistical Methods in Astrophysics

## Numerical Methods Topic IV. FFTs and their Applications

These notes are for the course Astronomy 8824: Numerical and Statistical Methods in Astrophysics. It is based on notes from David Weinberg with modifications and additions by Paul Martini.
David's original notes are available from his website: http://www.astronomy.ohio-state.edu/~dhw/A8824/index.html

#### Background reading: 
- Numerical Recipes Chapers 12 and 13, especially $\S\S 12.0-12.2, 13.0-13.2$
- _The Fourier Transform And Its Applications_ by Ronald Bracewell

In [1]:
import math
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy as sp

# matplotlib settings 
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('lines', linewidth=2)
plt.rc('axes', linewidth=2)
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)   # fontsize of the figure title

In [2]:
! pip install pyfftw
import pyfftw

LaTex macros hidden here -- 
$\newcommand{\expect}[1]{{\left\langle #1 \right\rangle}}$
$\newcommand{\intinf}{\int_{-\infty}^{\infty}}$
$\newcommand{\xbar}{\overline{x}}$
$\newcommand{\ybar}{\overline{y}}$
$\newcommand{\like}{{\cal L}}$
$\newcommand{\llike}{{\rm ln}{\cal L}}$
$\newcommand{\xhat}{\hat{x}}$
$\newcommand{\yhat}{\hat{y}}$
$\newcommand{\xhati}{\hat{x}_i}$
$\newcommand{\yhati}{\hat{y}_i}$
$\newcommand{\sigxi}{\sigma_{x,i}}$
$\newcommand{\sigyi}{\sigma_{y,i}}$
$\newcommand{\cij}{C_{ij}}$
$\newcommand{\cinvij}{C^{-1}_{ij}}$
$\newcommand{\cinvkl}{C^{-1}_{kl}}$
$\newcommand{\cinvmn}{C^{-1}_{mn}}$
$\newcommand{\valpha}{\vec \alpha}$
$\newcommand{\vth}{\vec \theta}$
$\newcommand{\ymod}{y_{\rm mod}}$
$\newcommand{\dy}{\Delta y}$
$\newcommand{tphi}{\widetilde{\phi}}$
$\newcommand{\trho}{\widetilde{\rho}}$


### Introduction and Motivation

Fourier Transforms are typically applied to two classes of problems: 1) As a useful computational tool, e.g. for N-body calculations; 2) As an analysis tool, e.g. to identify periodic signals. 

Physical descriptions are sometimes more natural in the Fourier domain, such as the distribution of density fluctuations, the characterization of turbulence, and the behaviour of variable objects.

Some mathematical operations are easier in the Fourier domain, or it may be possible to apply an approximate solution to modes of long or short wavelength. Examples are (de)convolution, correlation, optimal filtering, power spectrum analysis, computation of Fourier integrals, and solving some differential equations. 

Periodic signals are often interesting. Fourier Transforms can be an efficient way to look for them.

Some numerical operations can be done _much_ faster via Fast Fourier Transform if the number of data points $N$ is large.

For example, convolution is an operation $O(N^2)$ if done directly for two functions of length $N$.  (Sometimes the response function is much shorter than the function being convolved, in which case the scaling is only $O(N_{\rm data}N_{\rm response})$.)

But the FFT can be computed in $O(N \log_2 N)$ time, which may be orders-of-magnitude less.


### Definitions and Notation

I will mostly use the NR notation, but not always.

If $h(t)$ is a function of time, then its Fourier transform is
$$
H(f) = \int_{-\infty}^\infty h(t) e^{2\pi ift}dt.
$$
We can recover $h(t)$ from $H(f)$ by applying an inverse
Fourier transform,
$$
h(t) = \int_{-\infty}^\infty H(f) e^{-2\pi ift} df,
$$
i.e., just change the sign in the exponent. 

If $t$ is a time in sec, then $f$ is a frequency in sec$^{-1}$ (Hz).

It is useful to think of an FT as a linear operation on a function, and that $h(t)$ and $H(f)$ are two representations of the same function. 

Another common notation is to write the FT of $f(x)$ as
$$
\tilde{f}(k) = \int_{-\infty}^\infty f(x) e^{2\pi ikx}dx.
$$
This is common when $x$ is a unit of length, in which case 
the wavenumber $k$ has units of inverse length.

_Caution_: Other Fourier conventions are frequently used,
such as
$$
H(\omega) = \int_{-\infty}^\infty h(t) e^{i\omega t}dt, \qquad
h(t) = {1 \over 2\pi} \int_{-\infty}^\infty H(\omega) e^{-i\omega t}dt.
$$

The NR convention ($2\pi$ in the exponent) is standard for numerical
FTs, and it leaves you with less to remember.

However, you always have to check Fourier conventions when 
comparing to or using papers or books, and figuring out the
conversions can be a pain.

If you use Fourier transforms in a paper, make clear what convention
you are using.


### Decomposition and Identities

Recall that $e^{2\pi i ft} = \cos(2\pi ft) + i\sin(2\pi ft)$.

The FT relation expresses a decomposition of the function $h(t)$ into a sum (integral in the continuous case) of sinusoidal functions.

The sinusoidal basis is complete and orthogonal.  Any function can be decomposed this way, and different modes are independent, in that, for positive integers $m$, $n$:
$$\eqalign{
\int_{-\pi}^\pi \cos nx\cos mx dx &= \pi \delta_{mn}, \cr
\int_{-\pi}^\pi \sin nx\sin mx dx &= \pi \delta_{mn}, \cr
\int_{-\pi}^\pi \cos nx\sin mx dx &= 0.
}
$$

In general, a FT maps a complex function into another complex function.

If $h(t)$ is real, then
$$
H(f) = \int_{-\infty}^\infty \cos(2\pi ft) \,h(t) dt + 
       i \int_{-\infty}^\infty \sin(2\pi ft)\, h(t) dt
$$
is Hermitian, $H(f) = [H(-f)]^*$, because cosine is an even function and sine is an odd function. (Reminder: an even function is one where $f(t) = f(-t)$ and an odd function is one where $g(t) = -g(-t)$.)  

A Hermition Matrix is a complex, square matrix that is equal to its own conjugate transform. That is: $a_{ij} = \bar{a_{ji}}$ or $A = \bar{A^T}$. They are the complex extension of real, symetric matrices, and both always have real eigenvalues. 

If $h(t)$ is real and symmetric about zero then $H(f)$ is real (see NR for other symmetries). Symmetries can be useful because in some cases they may be used to improve computational efficiency.


#### Other useful identities:

If $h(t)$ and $H(f)$ are an FT pair then
$$\eqalign{
h(at)~ \hbox{is a Fourier transform pair with}~ {1\over|a|} H(f/a) ~  \mathrm{- (this\,is\,time\,scaling)} \cr
\frac{1}{|b|} h(t/b) ~ \hbox{is a Fourier transform pair with}~ H(bf) ~ \mathrm{- (this\,is\,frequency\,scaling)}
}$$
If a function is narrow in the time domain, then its Fourier transform is broad in the frequency domain, and vice versa.

For example (a useful one), with our Fourier convention, the FT of a time-domain Gaussian $e^{-t^2/2\sigma_t^2}$ is a frequency-domain Gaussian $e^{-2\pi^2\sigma_t^2 f^2} = e^{-f^2/2\sigma_f^2}$ with $\sigma_f=1/(2\pi\sigma_t)$.

With the $H(\omega)$ convention, $\sigma_\omega=1/\sigma_t$.

A shift of origin in the time domain is equivalent to shifting the complex phase in the Fourier domain,
$$
h(t-t_0)~ \hbox{is a Fourier transform pair with}~ H(f)e^{2\pi i ft_0}.
$$

The FT is a linear operation, so the FT of a linear combination of functions is the linear combination of their FTs.

### The Convolution Theorem

The convolution of two functions $g$ and $h$ is
$$
f(t) = g * h \equiv \int_{-\infty}^\infty g(\tau)h(t-\tau)d\tau.
$$

The FT of $g * h$ is $G(f) H(f)$, i.e., the Fourier transform of the convolution is the product of the Fourier transforms. 

### The Correlation Theorem
The correlation of two functions $g$ and $h$ is
$$
\mathrm{Corr} (g,h) \equiv \int_{-\infty}^\infty g^*(t)h(t+\tau)dt
$$
where $g^*(t)$ is the complex conjugate of $g(t)$ and $\tau$ is often called the lag. 

The FT of Corr($g,h$) is $G^*(f)H(f)$. In other words, multiplying the FT of a function by the complex conjugate of the FT of the other function is equal to their correlation.

Reminder that the complex conjugate of a complex number is another complex number with the opposite sign on the imaginary component, e.g. the complex conjugate of $5 + 3i$ is $5 - 3i$. 

### Nyquist frequency, aliasing

If a function is tabulated at discrete, evenly spaced locations with separation $\Delta$, then the highest frequency Fourier component that can be represented is one with critical frequency
$$
f_c = {1\over 2\Delta},
$$
known as the Nyquist frequency. For example, $\Delta$ could refer to discrete spatial or temporal samples of the signal.

A Nyquist frequency sine wave is sampled with two points per cycle.

If the function $h(t)$ is _bandwidth limited_, with the amplitude of all Fourier modes with $f > f_c$ equal to zero, then the sampled values at spacing $\Delta$ are enough to compute $h(t)$ exactly at any point using sinc interpolation (NR eq. 12.1.3).

$$
h(t) = \Delta \sum^{\infty}_{n=-\infty} h_n \frac{\sin [2 \pi f_c(t - n\Delta)]}{\pi (t - n\Delta) }
$$

This is a remarkable fact, and a useful one, e.g., for reconstructing a time series or shifting images to register
them _IF_ the PSF is fully sampled. This says that _all_ of the information content of a bandwidth-limited signal represented in this way. 

However, if it has power at $f>f_c$, then that power will spuriously contaminate Fourier modes with $f < f_c$.

This is known as aliasing.  

### The One Dimensional Fast Fourier Transform

Suppose that we have a function $h(t)$ tabulated on a grid of $N$ evenly spaced values.

$$ 
H(f_n) = \int^{\infty}_{-\infty} h(t) \e^{2 \pi i f_n t} dt \approx 
\Delta \sum^{N-1}_{k=0} h_k \e^{2 \pi i f_n t_k} 
$$

An FFT is an efficient algorithm for computing the _discrete Fourier transform_, the sum
$$
H_n \equiv \sum_{k=0}^{N-1} h_k e^{2\pi i \, kn/N},
$$
for all values $n=-N/2$ to $N/2$. The relation of the discrete to the continuous FT can be viewed as samples of the continuous function.

In this notation, $k$ is indexing time or position values; $n$ represents frequency or wavenumber.

Naively, it looks like computing this for all $n$ should be an $O(N^2)$ operation.

The FFT uses the fact that a discrete Fourier transform of length $N$ can be written as a sum of two discrete Fourier transforms of length $N/2$.

This rule can be applied recursively to turn computing the Fourier transform into an $O(N\log_2 N)$ operation, vastly more efficient.

In practice, instead of $n$ running from $-N/2$ to $N/2$, it runs from 0 to $N-1$.  The 0 value represents zero frequency $n=0$, the values $1 \leq n \leq N/2-1$ correspond to frequencies $0 < f < f_c$, the values $N/2+1 \leq n \leq N-1$ correspond to $-f_c < f < 0$, and the value $n=N/2$ corresponds to both $f=f_c$ and $f=-f_c$.  

Here $f_c = N/2$ is the critical or Nyquist frequency, the maximum frequency that can be represented by $N$ evenly spaced values of the function.

From the above equation, one can see that for $n=N/2$
$$
H_n = \sum h_k \left(e^{i\pi}\right)^k = \sum h_k (-1)^k 
    = \sum h_k \left(e^{-i\pi}\right)^k = H_{-n}.
$$

The discrete inverse Fourier transform is 
$$
h_k = {1 \over N} \sum_{n=0}^{N-1} H_ne^{-2\pi i \,kn/N}.
$$
A forward transform followed by an inverse transform returns the original data values $h_k$.

Some FFTs have a different normalization convention, so that there is no $1/N$ and a forward-then-inverse transform returns the original values multiplied by $N$. 

In general, be careful about normalization issues and about which order the data are stored in.  You will typically need to do some experimentation to make sure you have everything right.

### One-Dimensional Real-Valued FFT

An FFT transforms an array of complex numbers to another array of complex numbers.

If you want to Fourier transform an array of real numbers, you can just set the imaginary parts to zero before applying a regular FFT.

The outputs will be complex numbers, but they will satisfy the _Hermitian_ condition $H(f) = H^*(-f)$, so they still contain only $N$ pieces of information.

This strategy is fine if you are not strained for CPU time or memory, but it is clearly wasting some calculational energy, and it requires your array to be twice as long as it really needs to be.

A real-to-complex FFT routine takes a real array and returns the independent values of the Fourier transform.

This is especially valuable when you get to multi-dimensional FFTs, whose storage requirements scale as $N^{n_{\rm dim}}$.

### Example usages of np.fft

In [3]:
np.random.seed(1216)
x=np.random.random(8)
print("x = ", x)
y=np.fft.fft(x)
print("y = ", y)
z=np.fft.ifft(y)
print("z = ", z)
print("\n")
print("Note that while x is a real array, its FT y is complex. The FT of y is z. It is also complex, although with zero imaginary values.")

print("To just keep the real input, you may use np.fft.rfft() and np.fft.irfft() ")
y=np.fft.rfft(x)
print("y = ", y)
z=np.fft.irfft(y)
print("z = ", z)

x =  [0.99559818 0.62341091 0.87609369 0.88937786 0.38676248 0.6899199
 0.99285255 0.46804209]
y =  [ 5.92205765+0.j          0.26387737-0.13414157j -0.48658558+0.04408914j
  0.95379404-0.36765929j  0.58055615+0.j          0.95379404+0.36765929j
 -0.48658558-0.04408914j  0.26387737+0.13414157j]
z =  [0.99559818+0.j 0.62341091+0.j 0.87609369+0.j 0.88937786+0.j
 0.38676248+0.j 0.6899199 +0.j 0.99285255+0.j 0.46804209+0.j]


Note that while x is a real array, its FT y is complex. The FT of y is z. It is also complex, although with zero imaginary values.
To just keep the real input, you may use np.fft.rfft() and np.fft.irfft() 
y =  [ 5.92205765+0.j          0.26387737-0.13414157j -0.48658558+0.04408914j
  0.95379404-0.36765929j  0.58055615+0.j        ]
z =  [0.99559818 0.62341091 0.87609369 0.88937786 0.38676248 0.6899199
 0.99285255 0.46804209]


### Multi-dimensional FFT

A 2-dimensional FFT is
$$
H(n_1,n_2) \equiv \sum_{k_2=0}^{N_2-1}
                  \sum_{k_1=0}^{N_1-1}
		  e^{2\pi i \,k_2 n_2/N_2}
		  e^{2\pi i \,k_1 n_1/N_1}
		  h(k_1,k_2).
$$

N-dimensional FFTs can be computed by repeated applications of 1-d FFTs, with a fair amount of bookkeeping.

It is generally easiest to use a routine specifically written for multi-dimensional FFTs.

If you are starting with a real array, you should try to use a real-to-complex routine to save memory.

Pay careful attention to how the data are stored --- this may be different from one routine to another, and it is the
trickiest thing to figure out.

### Calculating gravitational accelerations by FFT

Computing gravitational potentials is an interesting special case of convolution.

Suppose we want to compute the gravitational forces in a system of $N$ bodies, where $N$ might be very large (for example $10^6-10^9$) to represent a galaxy or a large cosmological volume.

A naive calculation of
$$
\vec a_j = \sum_{i=1}^N - {Gm_i \vec r_{ij} \over r_{ij}^3}
$$
is $O(N^2)$ for $N$ particles.

Impossible to do large $N$.  Need more efficient method.

One approach:

$$\eqalign{
\nabla^2\phi(\vec r) &= 4\pi G\rho(\vec r) \cr
\phi(\vec r) &= \int \tphi (\vec k) e^{-2\pi i \vec k \cdot \vec r} d^3k,
}
$$
where
$$
\tphi(\vec k) = \int \phi (\vec r) e^{2\pi i \vec k \cdot \vec r} d^3r 
$$ 
is the Fourier transform of $\phi$.

$$\eqalign{
\vec\nabla\phi &= -2\pi i\vec k\int\tphi(\vec k)e^{-2\pi i\vec k\cdot\vec r}d^3k \cr
\nabla^2\phi &= -4\pi^2 k^2 \int\tphi(\vec k)e^{-2\pi i\vec k\cdot\vec r}d^3k.
}
$$

Hence $4\pi G\rho(r) = \nabla^2\phi(r)$ implies
$$
4\pi G\ \int \trho (\vec k) e^{-2\pi i \vec k\cdot\vec r} d^3k = 
-4\pi^2 k^2 \int\tphi(\vec k)e^{-2\pi i\vec k\cdot\vec r}d^3k,
$$
implying
$$
\tphi(\vec k) = -{G \over \pi k^2}\trho(\vec k).
$$

This can be a very efficient means of computing the gravitational potential because of numerically efficient Fourier transform routines.

#### Basic scheme:

Compute $\rho(x,y,z)$ on grid from particle distribution by interpolation.

Compute $\trho(k_x,k_y,k_z)$ by FFT, then $\tphi$ from above equation.

Inverse FFT to get $\phi(x,y,z)$.  

Numerically differentiate $\phi(x,y,z)$ to get accelerations.

There are various subtleties about how to do the differentiation, and about the best choice of "Green's function"
(which may not be simply $1/k^2$).

Forces are automatically softened (relative to $1/r^2$) on the scale of grid cell.

If this is adequate force resolution, then the FFT method is very efficient.

The FFT method naturally imposes a periodic boundary condition, which is sometimes desirable (e.g., a cosmological volume) and sometimes not (e.g., a galaxy merger).

### FFTW

The best library of routines for high-performance FFTs is probably `FFTW`, "The Fastest Fourier Transform in the West," freely available from www.fftw.org.

The FFTW routines analyze your machine to decide what algorithm will actually run fastest on your available hardware.

Equally important, the library includes multi-dimensional FFTs, real-to-complex FFTs, and works with any dimension of
array, not just powers of 2.  

You will probably get the best performance if you use array sizes that are multiples of powers of 2, 3, and 5.

There is a `pyFFTW` package that is a pythonic wrapper around FFWT such that they are called like the `numpy.fft` routines.

If you're working in python and want higher performance code, it is worthwhile to use `pyFFTW`. If not, the numpy or scipy implementations are straightforward and should already be available by default. In some tests I ran, I found the scipy implementation is actually almost as good as pyFFTW. Note also that performance differences are not too meaningful for small arrays. 

If you're working in C or Fortran, you may want to simply use the FFTW routines.

### Example usage and performance comparison of np.fft, sp.fftp, and FFTW

For a short intro and tutorial on pyFFTW see: https://hgomersall.github.io/pyFFTW/

In [4]:
# Create a random array for testing
np.random.seed(1216)
N = 2**18
vec = np.random.random(size=(N)).astype('float32')

%timeit -r 5 -n 256 np.fft.rfft(vec)

3.26 ms ± 15.6 µs per loop (mean ± std. dev. of 5 runs, 256 loops each)


In [5]:
# Compute the FFT with scipy 
%timeit -r 5 -n 256 sp.fft.rfft(vec)

2.12 ms ± 188 µs per loop (mean ± std. dev. of 5 runs, 256 loops each)


In [6]:
pyvec = pyfftw.byte_align(vec, dtype=vec.dtype)
%timeit  -r 5 -n 256 pyfftw.interfaces.numpy_fft.rfft(pyvec)

1.66 ms ± 74.6 µs per loop (mean ± std. dev. of 5 runs, 256 loops each)
